In [16]:
import functools, operator, requests, os, json
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain_community.chat_models.litellm import ChatLiteLLM
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI



In [17]:

llm = ChatOpenAI(model="gpt-4-turbo")

In [10]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage
)

@tool("web_search")
def web_search(query: str) -> str:
    """Search with Duck Duck Go by a query"""
    search = DuckDuckGoSearchRun()
    return search.run(query)

@tool("twitter_writer")
def write_tweet(content: str) -> str:
    """Based a piece of content, write a tweet."""
    chat = ChatLiteLLM(model="groq/Llama3-70b-8192")
    messages = [
      SystemMessage(
          content="You are a Twitter account operator."
                  " You are responsible for writing a tweet based on the content given."
                  " You should follow the Twitter policy and make sure each tweet has no more than 140 characters."
      ),
      HumanMessage(
          content=content
      ),
    ]
    response = chat(messages)
    return response.content

In [11]:
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

In [12]:
def create_agent(llm: ChatLiteLLM, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


In [18]:
members = ["Search_Engine", "Twitter_Writer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [19]:
search_engine_agent = create_agent(llm, [web_search], "You are a web search engine.")
search_engine_node = functools.partial(agent_node, agent=search_engine_agent, name="Search_Engine")

twitter_operator_agent = create_agent(llm, [write_tweet], "You are responsible for writing a tweet based on the content given.")
twitter_operator_node = functools.partial(agent_node, agent=twitter_operator_agent, name="Twitter_Writer")

workflow = StateGraph(AgentState)
workflow.add_node("Search_Engine", search_engine_node)
workflow.add_node("Twitter_Writer", twitter_operator_node)
workflow.add_node("supervisor", supervisor_chain)

In [20]:
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [21]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Write a tweet about LangChain news")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Search_Engine'}}
----
{'Search_Engine': {'messages': [HumanMessage(content="🚀 Exciting news from #LangChain! The startup has secured $25M in a Series A funding to expand its open-source framework for developing large language model apps. 🛠️ Plus, check out their new 'LangChain Experimental' for a leaner and safer experience. #TechNews #AI #OpenSource 🌐", name='Search_Engine')]}}
----
{'supervisor': {'next': 'Twitter_Writer'}}
----
{'Twitter_Writer': {'messages': [HumanMessage(content='"🚀 Breaking: #LangChain raises $25M in Series A funding to expand its open-source framework for large language model apps! 🛠️ Introducing \'LangChain Experimental\' for a leaner & safer experience. ✨ #TechNews #AI #OpenSource"', name='Twitter_Writer')]}}
----
{'supervisor': {'next': 'FINISH'}}
----
